In [14]:
# Modules needed
import requests
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Used to complete autocomplete in Jupyter Notebook
%config Completer.use_jedi = False



In [11]:
def webscrape_to_textfile():
    """Webscrapes the worldometers website for coronavirus statistics and saves information into textfile"""
    url = 'https://www.worldometers.info/coronavirus/countries-where-coronavirus-has-spread/'

    # gets the url's html
    page_content = requests.get(url)
    soup = BeautifulSoup(page_content.text, 'html.parser')

    web_data = []
    df = pd.DataFrame()
    
    
    # soup.find_all('td') will scrape all elements in the url's table elements
    data_iterator = iter(soup.find_all('td'))

    # Loop that repeats until there is no data left available for the iterator
    while True:
        try:
            country = next(data_iterator).text
            confirmed = next(data_iterator).text
            deaths = next(data_iterator).text
            continent = next(data_iterator).text
            
            new_row = {'country':country, 'confirmed':confirmed, 'deaths':deaths}
            print(new_row)
            # Replaces spaces with underscores in country and continent names to help with saving file content later
            country = country.replace(' ', '_')
            continent = continent.replace(' ', '_')
            # This just adds stats into the list while also replacing the confirmed and deaths into ints
            # NOTE: This creates a list of tuples
            df.append(new_row,ignore_index=True)
            
            web_data.append((country,
                             int(confirmed.replace(',', '')),  # This allows for numbers in millions
                             int(deaths.replace(',', '')),
                             continent
                             ))

        # StopIteration error occurs when there are no more elements to iterate through
        except StopIteration:
            break
    
    
    #  Sorts the data by number of confirmed cases
    web_data.sort(key=lambda row: row[1], reverse=True)
#     df.sort(key=lambda row: row[1], reverse=True)
    f = open('webscrape_data.txt', 'w')
    for tuple_unit in web_data:
        f.write(''.join(str(s) + ' ' for s in tuple_unit) + ' \n')
    print(df.head())
    f.close()
    
    return df

In [12]:
def webscrape_to_df():
    """Webscrapes the worldometers website for coronavirus statistics and saves information into textfile"""
    url = 'https://www.worldometers.info/coronavirus/countries-where-coronavirus-has-spread/'

    # gets the url's html
    page_content = requests.get(url)
    soup = BeautifulSoup(page_content.text, 'html.parser')

    df = pd.DataFrame()
    result = []
    
    # soup.find_all('td') will scrape all elements in the url's table elements
    data_iterator = iter(soup.find_all('td'))

    # Loop that repeats until there is no data left available for the iterator
    while True:
        try:
            country = next(data_iterator).text
            confirmed = next(data_iterator).text
            deaths = next(data_iterator).text
            continent = next(data_iterator).text
            
            # Replaces spaces with underscores in country and continent names to help with saving file content later
            country = country.replace(' ', '_')
            continent = continent.replace(' ', '_')
            
            new_row = {'Country':country, 'Confirmed Cases':confirmed, 'Deaths':deaths, 'Continent': continent}
#             print(new_row)
            result.append(new_row)
            # This just adds stats into the list while also replacing the confirmed and deaths into ints
            # NOTE: This creates a list of tuples
#             df.append(new_row,ignore_index=True)
            


        # StopIteration error occurs when there are no more elements to iterate through
        except StopIteration:
            break
            
    df = pd.DataFrame(result)           
    return df

In [15]:
temp = webscrape_to_df()
temp.head(10)

,Country,Confirmed Cases,Deaths,Continent
0,United_States,"34,540,922","620,256",North_America
1,India,"30,438,090","399,872",Asia
2,Brazil,"18,559,164","518,246",South_America
3,France,"5,775,301","111,082",Europe
4,Russia,"5,538,142","135,886",Europe
5,Turkey,"5,425,652","49,732",Asia
6,United_Kingdom,"4,828,463","128,162",Europe
7,Argentina,"4,470,374","94,304",South_America
8,Italy,"4,260,788","127,587",Europe
9,Colombia,"4,240,982","106,544",South_America
